# Sociation2Vec
Embedding-и слов аналогичные word2vec из базы ассоциаций [Sociation.org](http://sociation.org)

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm_notebook as progress
import gensim
import pickle

In [2]:
data_dir = "/Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus"
random_state = 777

%pylab inline
%config InlineBackend.figure_format = 'svg' 
%config InlineBackend.figure_format = 'retina'

# отключаем ворнинги
import warnings
warnings.simplefilter('ignore')

# дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 14,9

Populating the interactive namespace from numpy and matplotlib


### Лучший на данный момент вариант
 - corpus
 - normalize_matrix
 - ppmi (0.75, 5)
 - tfidf (по словам)
 - svd 800
 - normalize_matrix
 - normalize_l2 (при получении вектора слова)
 
```python
corpus, words_dict, assoc_dict = load_source_corpus()
corpus_csc = corpus_to_sparse(corpus)
corpus_csc = normalize_matrix(corpus_csc, with_mean=False)
corpus_ppmi = get_ppmi_weight(corpus_csc, cds=0.75, k=5)
tfidf, corpus_ppmi = get_tfidf(corpus_ppmi)
svd, corpus_ppmi_svd = get_svd(corpus_ppmi, 800)
corpus_ppmi_svd = normalize_matrix(corpus_ppmi_svd)
similarity_index = create_similarity_index(corpus_ppmi_svd)
sociation2vec = Sociation2Vec(svd, corpus_ppmi_svd, similarity_index, words_dict, assoc_dict)
```

## Вспомогательные функции

In [258]:
from scipy.sparse import dok_matrix, csr_matrix, save_npz, load_npz, issparse
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import scale


def get_csr_matrix_from_xyval_list(xyval_list, dtype=np.float32):
    """Создаёт CSR матрицу из списка [(x, y, value), ...]"""
    x, y, value = zip(*xyval_list)
    matrix = csr_matrix((value, (x, y)), dtype=dtype)
    matrix.eliminate_zeros()
    return matrix


def multiply_by_rows(matrix, row_coefs):
    """Умножает разреженную матрицу построчно на вектор"""
    normalizer = dok_matrix((len(row_coefs), len(row_coefs)))
    normalizer.setdiag(row_coefs)
    return normalizer.tocsr().dot(matrix)


def multiply_by_columns(matrix, col_coefs):
    """Умножает разреженную матрицу поколоночно на вектор"""
    normalizer = dok_matrix((len(col_coefs), len(col_coefs)))
    normalizer.setdiag(col_coefs)
    return matrix.dot(normalizer.tocsr())


def get_ppmi_weight(matrix, cds=0.75, k=5):
    """Рассчёт PPMI для разреженной матрицы"""
    marginal_rows = np.array(matrix.sum(axis=1))[:, 0]
    marginal_cols = np.array(matrix.sum(axis=0))[0, :] ** cds
    pmi = matrix * marginal_cols.sum()
    pmi = multiply_by_rows(pmi, np.reciprocal(marginal_rows))
    pmi = multiply_by_columns(pmi, np.reciprocal(marginal_cols))
    pmi.data = (np.log2(pmi.data) - np.log(k)).clip(0.0)
    return pmi


def get_svd(matrix, n_components, n_iter=10):
    """Факторизация матрицы c помощью SVD"""
    svd = TruncatedSVD(n_components=n_components, n_iter=n_iter, random_state=random_state)
    return svd, svd.fit_transform(matrix)


def get_nmf(matrix, n_components, alpha=0.1, l1_ratio=0.5):
    """Факторизация матрицы c помощью NMF"""
    nmf = NMF(n_components=n_components, alpha=alpha, l1_ratio=l1_ratio, random_state=random_state)
    return nmf, nmf.fit_transform(matrix)


def get_tfidf(matrix, smooth_idf=True, sublinear_tf=False):
    """
    Применение TfIdf к матрице, 
    TODO: попробовать оставить один IDF
    """
    tfidf = TfidfTransformer(smooth_idf=smooth_idf, sublinear_tf=sublinear_tf)
    return tfidf, tfidf.fit_transform(matrix)


def normalize_matrix(matrix, with_mean=True):
    """
    Нормализация матрицы (aka «стандартизация», -mean/std)
    Дважды транспонируется так как scale не хочет нормализовать разреженные матрицы по строкам
    """
    return scale(matrix.T, with_mean=with_mean).T

In [266]:
def corpus_to_sparse(corpus, words_count, assoc_count):
    return gensim.matutils.corpus2csc(corpus, num_terms=assoc_count, num_docs=words_count).T

def sparse_to_corpus(matrix):
    return gensim.matutils.Sparse2Corpus(matrix.T)

def dense_to_corpus(matrix):
    return gensim.matutils.Dense2Corpus(matrix.T)

In [267]:
def save_model(model, name):
    file_path = os.path.join(data_dir, name + ".pkl")
    print('Saving model to', file_path)
    with open(file_path, 'wb') as file:  
        pickle.dump(model, file)
        
def load_model(name):
    file_path = os.path.join(data_dir, name + ".pkl")
    print('Loading model from', file_path)
    with open(file_path, 'rb') as file:  
        return pickle.load(file)
    
def save_corpus(corpus, name):
    file_path = os.path.join(data_dir, name + ".npy")
    print('Saving corpus to', file_path)
    np.save(file_path, corpus)
        
def load_corpus(name):
    file_path = os.path.join(data_dir, name + ".npy")
    print('Loading corpus from', file_path)
    return np.load(file_path)

def create_similarity_index(corpus, name):
    if not gensim.matutils.isbow(corpus):
        corpus = dense_to_corpus(corpus)
    print("Creating similarity index...")
    similarity_index = gensim.similarities.MatrixSimilarity(corpus)
    file_path = os.path.join(data_dir, name + ".index")
    print('Saving similarity index to', file_path)
    similarity_index.save(file_path)
    return similarity_index

def load_similarity_index(name):
    file_path = os.path.join(data_dir, name + ".index")
    print('Loading similarity index from', file_path)
    return gensim.similarities.MatrixSimilarity.load(file_path)

## Загрузка словарей и корпуса ассоциаций

In [268]:
from sociation_corpus.utils import DictEncoder
from sociation_corpus.train_model import load_source_corpus

In [269]:
corpus, words_dict, assoc_dict = load_source_corpus(data_dir)
words_count = len(words_dict.decode)
assoc_count = len(assoc_dict.decode)
print("Слов: {}, Слов-ассоциаций: {}".format(words_count, assoc_count))

Loading source corpus...
Loading words index...
Loading assoc words index...
Слов: 29049, Слов-ассоциаций: 44342


In [261]:
from operator import itemgetter

def get_associations(word_name, count=None, corpus=corpus):
    """Ассоциации к слову"""
    associations = [
        (assoc_dict.decode[assoc_id], popularity)
        for assoc_id, popularity
        in sorted(corpus[words_dict.encode[word_name]], key=itemgetter(1), reverse=True)
    ]
    count = count or len(associations)
    return pd.DataFrame(associations[:count], columns=['word', 'popularity']).set_index('word')

In [262]:
get_associations('система').head()

,popularity
word,
структура,5.33333
государство,5.00000
порядок,4.81250
ниппель,3.00000
комплекс,2.00000


## Нормализация, PPMI, TF-IDF

In [271]:
corpus_csc = corpus_to_sparse(corpus, words_count, assoc_count)
corpus_csc.shape, (words_count, assoc_count)

((29049, 44342), (29049, 44342))

In [272]:
corpus_csc = normalize_matrix(corpus_csc, with_mean=False)

In [273]:
corpus_ppmi = get_ppmi_weight(corpus_csc, cds=0.75, k=5)
corpus_ppmi.shape

(29049, 44342)

In [274]:
tfidf, corpus_ppmi = get_tfidf(corpus_ppmi)
corpus_ppmi.shape

(29049, 44342)

In [275]:
get_associations('система', corpus=sparse_to_corpus(corpus_ppmi)).head(10)

,popularity
word,
симплекс-метод,0.175352
энергоснабжение,0.175352
подсистема,0.159927
операционка,0.154537
функциональность,0.145172
хря,0.128002
метаистория,0.123938
слом,0.119197
эмерджентность,0.116363


## Факторизация PPMI-матрицы с помощью SVD

In [19]:
# Загрузка корпуса и SVD модели
corpus_ppmi_svd = load_corpus('corpus_norm_ppmi_svd_norm_800')
svd = load_model('svd_800_norm_ppmi_model')

Loading corpus from /Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus/corpus_norm_ppmi_svd_norm_800.npy
Loading model from /Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus/svd_800_norm_ppmi_model.pkl


**ИЛИ**

In [21]:
# Создание и сохранение корпуса и SVD модели
svd, corpus_ppmi_svd = get_svd(corpus_ppmi, 800)
corpus_ppmi_svd = normalize_matrix(corpus_ppmi_svd)
save_corpus(corpus_ppmi_svd, 'corpus_norm_ppmi_svd_norm_800')
save_model(svd, 'svd_800_norm_ppmi_model')

Saving corpus to /Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus/corpus_norm_ppmi_svd_norm_800.npy
Saving model to /Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus/svd_800_norm_ppmi_model.pkl


In [188]:
corpus_ppmi_svd.shape

(29049, 800)

## Загрузка или создание/сохранение индекса

In [21]:
# Загрузка индекса
similarity_index = load_similarity_index('corpus_norm_ppmi_svd_norm_800')

Loading similarity index from /Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus/corpus_norm_ppmi_svd_norm_800.index


**ИЛИ**

In [277]:
# Создание и сохранение индекса
similarity_index = create_similarity_index(corpus_ppmi_svd, 'corpus_norm_ppmi_svd_norm_800')

Creating similarity index...
Saving similarity index to /Users/ur001/Documents/Sociation/Corpora/sociation_org_corpus/corpus_norm_ppmi_svd_norm_800.index


## Смотрим что вышло

In [190]:
def normalize_l2(vec):
    norm = np.linalg.norm(vec)
    if norm != 0: 
       vec = vec / norm
    return vec


def bow2nparray_vec(vec):
    return np.array(list(map(itemgetter(1), vec)))


def nparray2bow_vec(vec):
    return [(idx, val) for idx, val in enumerate(vec) if val]


class Sociation2Vec(object):
    """
    Поиск похожих слов по факторизованной матрице PPMI из ассоциаций с Sociation.org
    """
    def __init__(self, model_svd, corpus, similarity_index, words_dict, assoc_dict):
        self.model = model_svd
        self.corpus = corpus
        self.similarity_index = similarity_index
        self.words_dict = words_dict
        self.assoc_dict = assoc_dict

    def get_word_vector(self, word_name): 
        """
        Возвращает вектор слова
        Тут слово — это документ в терминах корпуса
        Слову соответствует вектор ассоциаций преобразованный в LSI-простраство
        :rtype: np.array
        """
        mul = -1 if word_name.startswith('-') else 1
        corpus_idx = self.words_dict.encode[word_name.strip('-')]
        word_vec = self.corpus[corpus_idx]
        return normalize_l2(word_vec * mul)
    
    def get_top_similar_to_words(self, word_names, count=15):
        """
        Выводит топ похожих слов на указанные
        Можно несколько через запятую и через минус: король,-мужчина,женщина = королева
        """
        # Должно быть так:
        # word_vec = nparray2bow_vec(sum(map(self.get_word_lsi_vector, word_names)))
        # Почему-то в глобальном `sum` оказался np.sum, и он не суммирует векторы как надо
        # По этому временно такой бред:
        vecs = list(map(self.get_word_vector, word_names))
        word_vec = vecs[0]
        for vec2 in vecs[1:]:
            word_vec += vec2
        word_vec = nparray2bow_vec(word_vec)
        
        return self.get_top_similar(word_vec, count, exclude=word_names)   
    
    def get_top_similar(self, word_vec, count=15, exclude=None):
        exclude = {self.words_dict.encode[item.strip('-')] for item in (exclude or [])}
        similarity = self.similarity_index[word_vec]
        top = sorted(enumerate(similarity), key=itemgetter(1), reverse=True)[:count + len(exclude)]
        return [
            (self.words_dict.decode[word_idx], similarity) 
            for word_idx, similarity in top
            if word_idx not in exclude
        ][:count]

In [191]:
sociation2vec = Sociation2Vec(svd, corpus_ppmi_svd, similarity_index, words_dict, assoc_dict)

In [192]:
def get_similar(word_names, sociation2vec=sociation2vec, count=10):
    """Ассоциации к слову"""
    word_names = word_names.split(',')
    similar = sociation2vec.get_top_similar_to_words(word_names, count=count)
    return pd.DataFrame(similar, columns=['word', 'similarity']).set_index('word')

In [279]:
get_similar('король,-мужчина,женщина')

,similarity
word,
королева,0.561213
корона,0.492371
царица,0.469780
королевство,0.457369
монархия,0.442670
трон,0.437980
императрица,0.429572
коронация,0.429451
монарх,0.426821


In [194]:
get_similar('король,-брат,сестра', count=1)

,similarity
word,
королева,0.624136


In [195]:
get_similar('король,-мальчик,девочка', count=1)

,similarity
word,
королева,0.565792


In [196]:
get_similar('ассоциация')

,similarity
word,
синоним,0.655544
гипоним,0.618039
аналогия,0.617473
схожесть,0.575375
гипероним,0.574386
подобие,0.571528
ассоциации,0.570339
похожесть,0.559030
сходство,0.552834


In [197]:
get_similar('жизнь', count=3)

,similarity
word,
бытие,0.523264
существование,0.512623
рождение,0.502476


In [32]:
get_similar('симбиоз')

,similarity
word,
кооперация,0.439639
товарищество,0.438475
ленточные черви,0.423492
объединение,0.419562
коллаборация,0.417079
сотрудничество,0.416681
дружба,0.411037
взаимопомощь,0.408255
генотип,0.394466


In [33]:
get_similar('система')

,similarity
word,
алгоритм,0.522618
программа,0.491287
структура,0.465371
операционная система,0.464559
настройки,0.454179
компьютер,0.441942
программирование,0.435987
упорядоченность,0.429573
программы,0.417259


In [34]:
get_similar('ватник,быдло', count=3)

,similarity
word,
гопник,0.629871
гопники,0.579116
гопота,0.568035


In [35]:
get_similar('ватник,-быдло', count=3)

,similarity
word,
телогрейка,0.427465
пуховик,0.368377
анорак,0.362613


In [36]:
get_similar('бесконечность')

,similarity
word,
вселенная,0.681268
безграничность,0.655359
вечность,0.650114
континуум,0.632230
петля мебиуса,0.620649
пространство,0.567535
непостижимость,0.547699
промежуток,0.537255
универсум,0.530941


In [37]:
get_similar('вечность', sociation2vec)

,similarity
word,
бесконечность,0.650114
время,0.585966
минута,0.547468
мгновение,0.530104
секунда,0.527109
промежуток,0.523073
длительность,0.521102
бытие,0.520548
континуум,0.493538


In [38]:
get_similar('плотность,объём', count=1)

,similarity
word,
масса,0.516677


In [39]:
get_similar('гид', count=4)

,similarity
word,
экскурсовод,0.722172
экскурсия,0.709419
достопримечательность,0.537955
путеводитель,0.522491


In [89]:
get_similar('кошка,-женщина,мужчина', count=4)

,similarity
word,
кот,0.534667
котик,0.442449
усы,0.430905
котёнок,0.415737


In [41]:
get_similar('мама,-женщина,мужчина', count=2)

,similarity
word,
папа,0.566477
отец,0.555301


In [42]:
get_similar('папа,-мужчина,женщина', count=2)

,similarity
word,
мама,0.596825
мать,0.590816


In [280]:
get_similar('сын,-мужчина,женщина', count=2)

,similarity
word,
мать,0.611202
дочь,0.606259


In [44]:
get_similar('дочь,-женщина,мужчина', count=2)

,similarity
word,
сын,0.555355
отец,0.530861


In [45]:
get_similar('брат,-мальчик,девочка', count=1)

,similarity
word,
сестра,0.607106


In [46]:
get_similar('принц,-мальчик,девочка', count=1)

,similarity
word,
принцесса,0.570067


In [47]:
get_similar('жена,-девочка,мальчик', count=2)

,similarity
word,
муж,0.594385
супруг,0.505211


In [48]:
get_similar('ворон,-мужчина,женщина', count=1)

,similarity
word,
ворона,0.396677


In [49]:
get_similar('волк,-мужчина,женщина', count=1)

,similarity
word,
волчица,0.43839


In [50]:
get_similar('собака,-женщина,мужчина', count=1)

,similarity
word,
пёс,0.603662


In [51]:
get_similar('бог,-христианство,индуизм', count=3)

,similarity
word,
шива,0.681705
брахма,0.675043
кришна,0.626828


In [52]:
get_similar('человек,-человечность')

,similarity
word,
дарвин,0.331385
горилла,0.323233
обезьяна,0.301998
естественный отбор,0.296974
бабуин,0.286413
макака,0.275529
существо,0.270807
особь,0.269833
примат,0.267564


In [53]:
get_similar('человек,человечность')

,similarity
word,
гуманность,0.583465
разум,0.526734
личность,0.523718
гуманизм,0.520462
индивид,0.513282
сущность,0.490149
человеколюбие,0.486066
душа,0.479412
доброта,0.472022


In [54]:
get_similar('помидоры,-красный цвет,зелёный цвет', count=2)

,similarity
word,
огурец,0.586347
зелень,0.575023


In [55]:
get_similar('огурцы,-зелёный цвет,красный цвет', count=2)

,similarity
word,
помидоры,0.512640
помидор,0.500788


In [56]:
get_similar('закат,-вечер,утро', count=3)

,similarity
word,
рассвет,0.691819
восход,0.655696
заря,0.633641


In [57]:
get_similar('свет,-день,ночь', count=1)

,similarity
word,
темнота,0.644368


In [58]:
get_similar('похолодание,-осень,весна', count=1)

,similarity
word,
оттепель,0.700926


In [59]:
get_similar('ключ,вода', count=1)

,similarity
word,
родник,0.592837


In [60]:
get_similar('ключ,болт', count=1)

,similarity
word,
гайка,0.798782


In [61]:
get_similar('ключ,криптография', count=2)

,similarity
word,
первичный ключ,0.716639
шифр,0.710854


In [62]:
get_similar('замок,ключ', count=2)

,similarity
word,
брелок,0.719851
отмычка,0.696581


In [63]:
get_similar('замок,средневековье', count=2)

,similarity
word,
замки,0.663148
рыцари,0.629686


In [64]:
get_similar('папа,мама', count=4)

,similarity
word,
отец,0.742033
мать,0.731331
родители,0.714131
родитель,0.689356


In [65]:
get_similar('квадрат', count=7)

,similarity
word,
прямоугольник,0.772479
параллелепипед,0.749473
треугольник,0.741927
четырёхугольник,0.741491
ромб,0.728660
геометрия,0.712658
углы,0.709029


In [66]:
get_similar('роберт шекли', count=10)

,similarity
word,
роджер желязны,0.814984
фантаст,0.739293
дуглас адамс,0.666545
айзек азимов,0.647544
станислав лем,0.644610
рэй брэдбери,0.605876
голова профессора доуэля,0.603078
лукьяненко,0.584134
александр беляев,0.576634


In [67]:
get_similar('тимур и его команда', count=10)

,similarity
word,
скауты,0.485325
детский писатель,0.461091
октябрята,0.429327
гайдар,0.391885
пионерский лагерь,0.389205
пионеры,0.385447
командообразование,0.376164
военная тайна,0.367057
тимур,0.352638


In [68]:
get_similar('инструментарий', count=10)

,similarity
word,
киянка,0.627503
клавикорд,0.624433
пассатижи,0.623281
зубрило,0.618656
гвоздодёр,0.584791
долото,0.568527
инструмент,0.566573
молоток,0.562166
напильник,0.541395


In [69]:
get_similar('медвежуть', count=10)

,similarity
word,
бурый цвет,0.399054
топтыгин,0.397614
мультяшка,0.364336
берлога,0.362444
всеядность,0.360414
норштейн,0.359966
бурый медведь,0.357576
топтун,0.352973
привидения,0.339674


In [70]:
get_similar('доброта')

,similarity
word,
добро,0.763855
сердечность,0.704085
милосердие,0.697351
добродетель,0.687927
добродушие,0.676433
сострадание,0.667714
человеколюбие,0.645881
великодушие,0.644186
доброжелательность,0.641481


In [71]:
get_similar('безвозмездность', count=3)

,similarity
word,
бескорыстность,0.665653
благотворительность,0.633934
бескорыстие,0.626533


In [72]:
get_similar('странность', count=5)

,similarity
word,
необычность,0.534394
неординарность,0.496574
уникальность,0.463565
нестандартность,0.454107
специфичность,0.431959


In [73]:
get_similar('женщина,девушка,-мужчина,-мужик', count=2)

,similarity
word,
красавица,0.427106
красотка,0.419579


In [74]:
get_similar('мужчина,мужик,-женщина,-девушка', count=3)

,similarity
word,
мужлан,0.353025
альфа-самец,0.333147
мачо,0.313318


In [75]:
get_similar('соль', count=5)

,similarity
word,
перец,0.499872
фа,0.499244
ре,0.489227
ля,0.473073
си,0.466800


In [76]:
get_similar('соль,-нота')

,similarity
word,
перец,0.404399
горчица,0.375945
уксус,0.373835
приправа,0.373037
паприка,0.350440
аджика,0.345090
рассол,0.344369
маринад,0.337871
чеснок,0.337394


In [170]:
get_similar('рикки-тикки-тави', count=3)

,similarity
word,
мангуст,0.705134
маугли,0.557264
киплинг,0.526887


In [171]:
get_similar('фотосессия', count=3)

,similarity
word,
фотограф,0.657098
позирование,0.599602
фотография,0.533045


In [198]:
get_similar('научный сотрудник')

,similarity
word,
учёная степень,0.776008
аспирант,0.745037
доцент,0.735436
академик,0.714776
доктор наук,0.684844
диссертация,0.684797
академгородок,0.636815
магистр,0.626043
учёный совет,0.620870


In [246]:
get_similar('пуговка,-пуговица,крышка', count=1)

,similarity
word,
крышечка,0.492089


In [242]:
get_similar('коробочка,-коробка,пуговка,-пуговица,носик,-нос,пёсик,-пёс,медведь', count=1)

,similarity
word,
медвежонок,0.34249


In [243]:
get_similar('коробочка,-коробка,пуговка,-пуговица,носик,-нос,пёсик,-пёс,мышь', count=1)

,similarity
word,
мышонок,0.451238


In [284]:
get_similar('фифа', count=5)

,similarity
word,
пенальти,0.628970
голкипер,0.615618
кержаков,0.594990
нападающий,0.590477
гол,0.589090


In [283]:
get_similar('фифа,-футбол', count=5)

,similarity
word,
фря,0.312235
цаца,0.284392
рыцарский поединок,0.280962
фиона,0.277079
герцогиня,0.260875


In [285]:
get_similar('няша', count=5)

,similarity
word,
няшность,0.724274
мимимишечки,0.583773
котейка,0.570982
котик,0.542336
заинька,0.540596
